In [1]:
import time
import csv
from IPython.display import display, Markdown, Latex
import networkx as nx
import bravo.regulation as bravo_reg
import pyBravo as bravo_main

def read_input_genes(filename):
    res = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in reader:
            res.append(''.join(row))
    return res

## Upstream regulation network reconstruction

This call would test the regulation network reconstruction on all available data sources, and stop at depth 2.  
`reconstructed_network = bravo.upstream_regulation(['JUN/FOS', 'SCN5A'], max_depth=2)`

This call would test the regulation network reconstruction on `reactome` and `msigdb` data sources, stopping when no new regulations can be found.   
`reconstructed_network = bravo.upstream_regulation(['JUN/FOS', 'SCN5A'], data_sources=['reactome', 'msigdb'])`

In [24]:
start_time = time.time()

# global options
bravo_reg.DECOMPOSE_COMPLEXES = True
bravo_reg.EXTEND_WITH_SYNONYMS = True
bravo_reg.EXTEND_WITH_SUFFIXES = True
bravo_reg.VERBOSE = False

""" all possible data sources """
ds = ['bind', 'biogrid', 'corum',
                'ctd', 'dip', 'drugbank', 'hprd', 'humancyc', 'inoh',
                'intact', 'kegg', 'mirtarbase', 'netpath', 'panther',
                'pid', 'psp', 'reactome', 'reconx', 'smpdb', 'wp',
                'intact_complex', 'msigdb']

""" removing mirtarbase """
target_data_sources = set(ds) - set(['mirtarbase'])

""" specifiyng inputs """
gene_list = ['JUN/FOS', 'SCN5A']
#gene_list = read_input_genes('iLiverCancer1715.target')

""" reconstruction call """
reconstructed_network = bravo_reg.upstream_regulation(gene_list, 
                                                      data_sources=target_data_sources, 
                                                      max_depth=1, 
                                                      already_explored = [], 
                                                      sif_network = [])

elapsed_time = round((time.time() - start_time), 2)

print("--- Upstream regulation network in %s seconds ---" % elapsed_time)


exploration depth 0
to be explored ['JUN/FOS', 'SCN5A']

JUN/FOS decomposed into ['JUN', 'FOS']
to be explored after complex decomposition ['JUN/FOS', 'SCN5A', 'JUN', 'FOS']
new synonmys to be explored:['CDCD2', 'CMD1E', 'CMPD2', 'HB1', 'HB2', 'HBBD', 'HH1', 'ICCD', 'IVF', 'LQT3', 'Nav1.5', 'PFHB1', 'SSS1', 'VF1', 'AP-1', 'AP1', 'c-Jun', 'p39', 'AP-1', 'C-FOS', 'p55']
exploring ['JUN/FOS', 'SCN5A', 'JUN', 'FOS', 'CDCD2', 'CMD1E', 'CMPD2', 'HB1', 'HB2', 'HBBD', 'HH1', 'ICCD', 'IVF', 'LQT3', 'Nav1.5', 'PFHB1', 'SSS1', 'VF1', 'AP-1', 'AP1', 'c-Jun', 'p39', 'C-FOS', 'p55', 'JUN/FOS mRna', 'JUN/FOS protein', 'SCN5A mRna', 'SCN5A protein', 'JUN mRna', 'JUN protein', 'FOS mRna', 'FOS protein', 'CDCD2 mRna', 'CDCD2 protein', 'CMD1E mRna', 'CMD1E protein', 'CMPD2 mRna', 'CMPD2 protein', 'HB1 mRna', 'HB1 protein', 'HB2 mRna', 'HB2 protein', 'HBBD mRna', 'HBBD protein', 'HH1 mRna', 'HH1 protein', 'ICCD mRna', 'ICCD protein', 'IVF mRna', 'IVF protein']
		removing suffix  complex for NSC 689534/Co

## Metrics and SIF export of the regulation network

In [26]:
G = bravo_main.build_nx_digraph(reconstructed_network)

Number of nodes = 67
Number of edges = 68


In [27]:
G_unified = bravo_reg.fast_reg_network_unification(G, bravo_reg.index_syn)
print('Nodes after synonym-based unification = ' + str(len(G_unified.nodes())))
print('Edges after synonym-based unification = ' + str(len(G_unified.edges())))

		merging node ATF2 into node GDNF
		merging node ATF1 into node GDNF
		merging node c-Jun into node JUN
		merging node AP1 into node JUN
		merging node c-Jun into node JUN
		merging node AP1 into node JUN
		merging node ATF2 into node GDNF
		merging node ATF1 into node GDNF
Nodes after synonym-based unification = 63
Edges after synonym-based unification = 66


In [28]:
md = bravo_main.get_centrality_as_md(G_unified)
display(Markdown(md))


| Gene | Degree Centrality |
|------|------|
| FOS | 0.903 | 
| JUN | 0.177 | 
| JUN/FOS | 0.048 | 
| ERK1-2-active | 0.032 | 
| GDNF | 0.032 | 
| SMAD3/SMAD4/JUN/FOS | 0.016 | 
| JDP2 | 0.016 | 
| MIR101A | 0.016 | 
| ERK1-2/ELK1 | 0.016 | 
| Organoselenium Compounds analog/Copper | 0.016 | 


In [29]:
bravo_main.write_to_SIF(G_unified, 'tutorial.sif')
bravo_main.write_provenance(G_unified, 'tutorial-prov.csv')

SIF network written to tutorial.sif
Basic regulation reaction provenance written to tutorial-prov.csv
